In [2]:
from torch_geometric.data import Data
from transformers import AutoTokenizer, AutoModel
import spacy
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
import pickle
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import spacy
import torch
from transformers import AutoTokenizer, AutoModel
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import torch
from transformers import AutoTokenizer, AutoModel
import spacy

In [2]:
!pip install torch-geometric

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 49.0 MB/s eta 0:00:00


In [2]:
x = "UCHAMP , T., AND P OPOVI Ć , Z. 2002. Interactive skeleton-driven dynamic deformations. ACM Transactions on Graphics 21, 3, 586–593. C ATMULL , E. E. 1972. A system for computer generated movies. In Proc. ACM Annual Conf. August, 422–431. F REEMAN , W. T., AND T ENENBAUM , J. B. 1997. Learning bilinear models for two-factor problems in vision. In IEEE Computer Vision and Pattern Recognition. J AMES , D. L., AND P AI , D. K. 2002. DyRT: Dynamic response textures for real time deformation simulation with graphics hardware. ACM Transactions on Graphics 21, 3, 582–585. K RY , P. G., J AMES , D. L., AND P AI , D. K. 2002. Eigenskin: Real time large deformation character skinning in hardware. In ACM SIGGRAPH Symposium on Computer Animation, 153–160. L EWIS , J. P., C ORDNER , M., AND F ONG , N. 2000. Pose space deformations: A unified approach to shape interpolation and skeleton-driven deformation. In Proceedings of ACM SIGGRAPH 2000, Annual Conference Series, ACM SIGGRAPH. M AGNENAT -T HALMANN , N., L APERRIRE , R., AND T HALMANN , D. 1988. Jointdependent local deformations for hand animation and object grasping. In Proceedings of Graphics Interface ’88, 26–33. M ALANDAIN , G., AND B OISSONNAT , J.-D. 2002. Computing the diameter of a point set. In Discrete Geometry for Computer Imagery (DGCI 2002), A. Braquelaire, J.O. Lachaud, and A. Vialard, Eds., vol. 2301. N EBEL , J.-C., AND S IBIRYAKOV , A. 2002. Range flow from"

In [3]:
from torch_geometric.data import Data
from transformers import AutoTokenizer, AutoModel
import spacy
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel

from transformers import logging as transformers_logging

# Suppress Transformers warnings
transformers_logging.set_verbosity_error()

#Layer 1: Token level, creates dependency matrix with method --> create_dependency_matrix()

def create_dependency_matrix(doc, exclude_tags=["punct"]):
    """
    Create an adjacency matrix based on SpaCy's dependency parsing.

    Parameters:
        doc (spacy.tokens.Doc): Parsed SpaCy Doc object.
        exclude_tags (list): List of dependency tags to exclude.

    Returns:
        np.ndarray: Adjacency matrix of shape (num_tokens, num_tokens).
    """
    num_tokens = len(doc)
    matrix = np.zeros((num_tokens, num_tokens))

    for token in doc:
        # Only include valid dependencies
        if token.dep_ not in exclude_tags and token.head != token:
            matrix[token.i, token.head.i] = 1  # token -> head
            matrix[token.head.i, token.i] = 1  # head -> token

    return matrix



def get_scibert_embeddings(tokens):
    """
    Extract SciBERT embeddings for tokens in the input text.

    Parameters:
        tokens (list): List of tokens from the input text.

    Returns:
        torch.Tensor: Token embeddings of shape (num_tokens, embedding_dim).
        list: List of tokens (aligned with embeddings).
    """
    # Load SciBERT tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
    model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

    # Tokenize using the token list
    inputs = tokenizer(
        tokens,
        is_split_into_words=True,  # Preserve token order
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )

    # Map subword tokens back to full tokens
    word_ids = inputs.word_ids()

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    subword_embeddings = outputs.last_hidden_state.squeeze(0)

    # Aggregate subword embeddings to form word-level embeddings
    word_embeddings = []
    for i in range(len(tokens)):
        token_indices = [idx for idx, word_id in enumerate(word_ids) if word_id == i]
        if token_indices:
            word_embedding = subword_embeddings[token_indices].mean(dim=0)
            word_embeddings.append(word_embedding)

    word_embeddings = torch.stack(word_embeddings)  # Shape: (num_tokens, embedding_dim)
    return word_embeddings, tokens

def layer1_output(text,nlp):
    """
    Process text to generate SciBERT embeddings and dependency matrix for Layer 1.

    Parameters:
        text (str): Input text.

    Returns:
        torch.Tensor: SciBERT embeddings of shape (num_tokens, embedding_dim).
        np.ndarray: Dependency matrix of shape (num_tokens, num_tokens).
    """
    # Step 1: Parse text with SpaCy
    doc = nlp(text)

    # Step 2: Create dependency matrix
    dependency_matrix = create_dependency_matrix(doc)

    # Step 3: Extract tokens and SciBERT embeddings
    tokens = [token.text for token in doc]  # Use SpaCy tokens for consistency
    embeddings, scibert_tokens = get_scibert_embeddings(tokens)

    # Step 4: Validate alignment
    assert tokens == scibert_tokens, "Token mismatch between SpaCy and SciBERT!"

    return embeddings, dependency_matrix, tokens

# # Define Functions
# def get_scibert_embeddings(text):
#     """
#     Extract SciBERT embeddings for tokens in the input text.

#     Parameters:
#         text (str): Input text.

#     Returns:
#         torch.Tensor: Token embeddings of shape (num_tokens, embedding_dim).
#         list: List of tokens from the input text.
#     """
#     tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
#     model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

#     # Tokenize and encode
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
#     tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

#     with torch.no_grad():
#         outputs = model(**inputs)
#     embeddings = outputs.last_hidden_state.squeeze(0)  # Shape: (num_tokens, embedding_dim)

#     return embeddings, tokens


# def create_dependency_matrix(doc, exclude_tags=["punct"]):
#     """
#     Create an adjacency matrix based on SpaCy's dependency parsing.

#     Parameters:
#         doc (spacy.tokens.Doc): Parsed SpaCy Doc object.
#         exclude_tags (list): List of dependency tags to exclude.

#     Returns:
#         np.ndarray: Adjacency matrix of shape (num_tokens, num_tokens).
#     """
#     num_tokens = len(doc)
#     matrix = np.zeros((num_tokens, num_tokens))

#     for token in doc:
#         if token.dep_ not in exclude_tags and token.head != token:  # Exclude self-loops and specific tags
#             matrix[token.i, token.head.i] = 1  # token -> head
#             matrix[token.head.i, token.i] = 1  # head -> token

#     return matrix



# #NOT USING THIS
# def create_similarity_matrix(embeddings, threshold=0.85):
#     """
#     Create an adjacency matrix based on semantic similarity.

#     Parameters:
#         embeddings (torch.Tensor): Token embeddings of shape (num_tokens, embedding_dim).
#         threshold (float): Similarity threshold for creating edges.

#     Returns:
#         np.ndarray: Adjacency matrix of shape (num_tokens, num_tokens).
#     """
#     from sklearn.metrics.pairwise import cosine_similarity

#     # Convert embeddings to numpy
#     embeddings_np = embeddings.detach().cpu().numpy()

#     # Compute cosine similarity
#     similarity = cosine_similarity(embeddings_np)

#     # Threshold to create edges
#     matrix = (similarity > threshold).astype(float)
#     np.fill_diagonal(matrix, 0)  # Remove self-loops
#     return matrix


def create_graph_data(embeddings, adjacency_matrix):
    """
    Create a PyTorch Geometric Data object.

    Parameters:
        embeddings (torch.Tensor): Node features of shape (num_tokens, embedding_dim).
        adjacency_matrix (np.ndarray): Adjacency matrix of shape (num_tokens, num_tokens).

    Returns:
        Data: PyTorch Geometric Data object.
    """
    # Convert adjacency matrix to edge index
    edge_index = torch.tensor(np.nonzero(adjacency_matrix), dtype=torch.long)
    x = embeddings  # Node features are token embeddings
    return Data(x=x, edge_index=edge_index)


def layer_1(text,nlp):
    # Process layer 1
    #text = """We propose a layered framework for incorporating example-based skinning algorithms such as Pose Space Deformation or Shape-by-Example into an existing character animation system. The challenge in implementing example-based skinning in an existing system lies in the fact that it is generally believed that the interpolation of the examples is best performed before doing other skinning deformations (although there has been no analysis as to why this is the case), whereas the examples are specified by the user after the other deformations are performed. It is therefore necessary to invert the operation of these skinning and deformation operators. Existing systems typically allow layering of both basic skinning methods such as Skeleton Subspace Deformation (SSD) and other deformations such as lattices, etc., and commercial systems may further allow additional proprietary deformation algorithms as part of the character skinning. Unfortunately, understanding and accessing their various parameters can be laborious at best, and we do not have access to the algorithms in the case of commercial packages. The contributions of this paper are 1) a detailed analysis showing how inverting the skinning operations leads to better example interpolation, and 2) a demonstration"""
    assert len(text) > 5, "Tokens are too short"
    embeddings, dependency_matrix, tokens = layer1_output(text,nlp)


    # create graph data
    #graph_data = create_graph_data(embeddings, dependency_matrix)
    return embeddings, dependency_matrix



## TESTING THE DEPENDENCY PARSER - uncomment to text
# if __name__ == "__main__":
#     text = "The cat sat on the mat."
#     #text = "The quick brown fox jumps over the lazy dog."

#     # Step 1: Parse text with SpaCy
#     nlp = spacy.load("en_core_web_sm")
#     doc = nlp(text)

#     # Step 2: Create the dependency matrix
#     dependency_matrix = create_dependency_matrix(doc)

#     # Step 3: Extract tokens and SciBERT embeddings
#     tokens = [token.text for token in doc]  # All tokens, including punctuation
#     embeddings, scibert_tokens = get_scibert_embeddings(tokens)

#     for token in doc:
#         print(f"Token: {token.text}, Head: {token.head.text}, Dependency: {token.dep_}, Index: {token.i}, Head Index: {token.head.i}")

#     # Ensure alignment
#     assert tokens == scibert_tokens, "Token mismatch between SpaCy and SciBERT!"

#     # Print results
#     print("Tokens:", tokens)
#     print("Embeddings Shape:", embeddings.shape)
#     print("Dependency Matrix:")
#     print(dependency_matrix)



In [4]:
#Layer 3:
#creates concept-level adjacency matrix with  method -->  layer_3_concept_level()

def prioritize_named_entities(text,nlp):
    """
    Extract and prioritize named entities over noun chunks.

    Parameters:
        text (str): Input text.

    Returns:
        list: Prioritized list of meaningful concepts.
    """
    
    doc = nlp(text)

    # Extract named entities
    named_entities = [ent.text for ent in doc.ents]

    # Extract longer noun chunks (e.g., more than two words)
    noun_chunks = [chunk.text for chunk in doc.noun_chunks if len(chunk.text.split()) > 2]

    # Combine and deduplicate concepts
    concepts = list(set(named_entities + noun_chunks))
    return concepts




def get_concept_embeddings(concepts):
    """
    Generate embeddings for a list of concepts using SciBERT.

    Parameters:
        concepts (list): List of concepts (strings).

    Returns:
        torch.Tensor: Concept embeddings of shape (num_concepts, embedding_dim).
    """
    tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
    model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

    concept_embeddings = []
    for concept in concepts:
        inputs = tokenizer(concept, return_tensors="pt", truncation=True, padding=True)
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token for the concept
        concept_embeddings.append(cls_embedding.squeeze(0))

    return torch.stack(concept_embeddings)


def create_concept_level_matrix(concept_embeddings, threshold=0.85):
    """
    Create adjacency matrix for concept-level relationships.

    Parameters:
        concept_embeddings (torch.Tensor): Concept embeddings of shape (num_concepts, embedding_dim).
        threshold (float): Similarity threshold for connecting concepts.

    Returns:
        np.ndarray: Adjacency matrix of shape (num_concepts, num_concepts).
    """
    embeddings_np = np.array(concept_embeddings.detach().cpu().tolist())
    similarity = cosine_similarity(embeddings_np)

    adjacency_matrix = (similarity > threshold).astype(float)
    np.fill_diagonal(adjacency_matrix, 0)  # Remove self-loops

    # # Dynamically lower threshold if matrix is empty
    # if not np.any(adjacency_matrix):
    #     print("Warning: No connections found. Lowering similarity threshold to 0.7.")
    #     adjacency_matrix = (similarity > 0.75).astype(float)
    #     np.fill_diagonal(adjacency_matrix, 0)

    return adjacency_matrix

def add_cooccurrence_edges(concepts, adjacency_matrix, text,nlp):
    doc = nlp(text)

    # Build a map of sentences and the concepts they contain
    sentence_concepts = {i: [] for i, sent in enumerate(doc.sents)}
    for i, sent in enumerate(doc.sents):
        for concept in concepts:
            if concept in sent.text:
                sentence_concepts[i].append(concept)

    # Add co-occurrence edges without overwriting existing edges
    for sentence, sentence_concept_list in sentence_concepts.items():
        for i, concept1 in enumerate(sentence_concept_list):
            for concept2 in sentence_concept_list[i + 1:]:
                idx1, idx2 = concepts.index(concept1), concepts.index(concept2)
                if adjacency_matrix[idx1, idx2] == 0:  # Only add if no existing edge
                    adjacency_matrix[idx1, idx2] = 1
                    adjacency_matrix[idx2, idx1] = 1  # Bidirectional

    return adjacency_matrix


def layer_3_concept_level(text, nlp, threshold=0.85):
    """
    Process text to generate concept embeddings and adjacency matrix.

    Parameters:
        text (str): Input text.
        threshold (float): Similarity threshold for creating edges.

    Returns:
        torch.Tensor: Concept embeddings of shape (num_concepts, embedding_dim).
        np.ndarray: Adjacency matrix of shape (num_concepts, num_concepts).
        list: Filtered concepts.
    """
    # Step 1: Extract and prioritize named entities and noun chunks
    concepts = prioritize_named_entities(text,nlp)

    # Step 2: Filter out generic or irrelevant concepts
    # concepts = filter_concepts(concepts)

    # Step 3: Generate concept embeddings
    concept_embeddings = get_concept_embeddings(concepts)

    # Step 4: Create adjacency matrix for semantic similarity
    concept_matrix = create_concept_level_matrix(concept_embeddings, threshold=threshold)

    # Step 5: Add co-occurrence-based edges
    concept_matrix = add_cooccurrence_edges(concepts, concept_matrix, text,nlp)

    return concept_embeddings, concept_matrix, concepts


#NOTE: the current concept graph is not catching every concept such as "Shape-by-Example"
def visualize_concept_graph(concepts, matrix):
    G = nx.Graph()
    for i, concept in enumerate(concepts):
        G.add_node(i, label=concept)

    for i in range(len(matrix)):
        for j in range(i + 1, len(matrix)):
            if matrix[i, j] > 0:
                G.add_edge(i, j)

    pos = nx.spring_layout(G)
    nx.draw(G, pos, with_labels=True, labels={i: concepts[i] for i in range(len(concepts))}, node_size=5000, font_size=10)
    plt.title("Concept-Level Graph")
    plt.show()



# if __name__ == "__main__":
#     # Input text
#     #text = """We propose a layered framework for incorporating example-based skinning algorithms such as Pose Space Deformation or Shape-by-Example into an existing character animation system."""

#     # text = """We propose a layered framework for incorporating example-based skinning algorithms such as Pose Space Deformation or Shape-by-Example into an existing character animation system.
#     # The challenge in implementing example-based skinning in an existing system lies in the fact that it is generally believed that the interpolation of the examples is best performed before doing other skinning deformations."""

#     #NOTE: the concept graph is not catching every concept such as "Shape-by-Example"
#     text = """We propose a layered framework for incorporating example-based skinning algorithms such as Pose Space Deformation or Shape-by-Example into an existing character animation system.
#     The challenge in implementing example-based skinning in an existing system lies in the fact that it is generally believed that the interpolation of the examples is best performed before doing other skinning deformations.
#     It is therefore necessary to invert the operation of these skinning and deformation operators."""
#     # Process Layer 3
#     concept_embeddings, concept_matrix, concepts = layer_3_concept_level(text)

#     # Print results
#     print(f"Number of concepts: {len(concepts)}")
#     print(" Concepts:", concepts)
#     print("Concept-Level Adjacency Matrix:")
#     print(concept_matrix)

#     # Visualize the graph
#     visualize_concept_graph(concepts, concept_matrix)

In [5]:
#LAYER 2:
# Get sentence-level adjacency matrix with method --> layer_2_sentence_level


def get_sentences(text,nlp):
    """
    Split text into sentences using SpaCy.

    Parameters:
        text (str): Input text.

    Returns:
        list: List of sentences.
    """
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    return sentences

#generaate sentence emnbeddings w CLS token from scibert

def get_sentence_embeddings(sentences):
    """
    Generate sentence embeddings using the CLS token from SciBERT.

    Parameters:
        sentences (list): List of sentences.

    Returns:
        torch.Tensor: Sentence embeddings of shape (num_sentences, embedding_dim).
    """
    tokenizer = AutoTokenizer.from_pretrained("allenai/scibert_scivocab_uncased")
    model = AutoModel.from_pretrained("allenai/scibert_scivocab_uncased")

    embeddings = []
    for sentence in sentences:
        # Tokenize and encode each sentence
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)

        # Extract CLS token embedding
        with torch.no_grad():
            outputs = model(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token is at index 0
        embeddings.append(cls_embedding.squeeze(0))

    return torch.stack(embeddings)

    #centence-level adjacency matric w/ semtantic similarity between sentences & sequetial edges


def create_sentence_level_matrix(sentence_embeddings, threshold=0.85):
    """
    Create an adjacency matrix for sentence-level relationships.

    Parameters:
        sentence_embeddings (torch.Tensor): Sentence embeddings of shape (num_sentences, embedding_dim).
        threshold (float): Similarity threshold for connecting sentences.

    Returns:
        np.ndarray: Adjacency matrix of shape (num_sentences, num_sentences).
    """

    #DIDNT WORK ON MY SYSTEM: change to this if works on yours
    #embeddings_np = sentence_embeddings.detach().cpu().numpy()
    #COMMENT LINE BELOW IF THE ABOVE WORKS
    embeddings_np = np.array(sentence_embeddings.detach().cpu().tolist())

    # Compute cosine similarity
    similarity = cosine_similarity(embeddings_np)

    # Create adjacency matrix based on similarity
    adjacency_matrix = (similarity > threshold).astype(float)

    # Add sequential connections
    num_sentences = embeddings_np.shape[0]
    for i in range(num_sentences - 1):
        adjacency_matrix[i, i + 1] = 1
        adjacency_matrix[i + 1, i] = 1

    np.fill_diagonal(adjacency_matrix, 0)  # Remove self-loops
    return adjacency_matrix



    #combine
def layer_2_sentence_level(text,nlp):
    """
    Process text to generate sentence embeddings and adjacency matrix.

    Parameters:
        text (str): Input text.

    Returns:
        torch.Tensor: Sentence embeddings of shape (num_sentences, embedding_dim).
        np.ndarray: Adjacency matrix of shape (num_sentences, num_sentences).
    """
    # Step 1: Tokenize text into sentences
    sentences = get_sentences(text,nlp)

    # Step 2: Generate sentence embeddings
    sentence_embeddings = get_sentence_embeddings(sentences)

    # Step 3: Create sentence-level adjacency matrix
    sentence_matrix = create_sentence_level_matrix(sentence_embeddings)

    masked_results = []
    for mask in range(len(sentences)):
        try:
            embedding = sentence_embeddings[mask,:]
            remaining_embeddings = torch.cat((sentence_embeddings[:mask,:], sentence_embeddings[mask+1:,:]), dim=0)
            #print(embedding)

            sentence = sentences[mask]
            assert len(sentence) > 0, "torch is too small"
        
            masked_sentence_matrix = create_sentence_level_matrix(remaining_embeddings)

            masked_results.append([[remaining_embeddings,masked_sentence_matrix],[sentence, embedding]])
        except Exception as e:
            print(f"lost mask {e}")

    return sentence_embeddings, sentence_matrix, masked_results


# #TESTING
# if __name__ == "__main__":
#     #First three sentences of the abstract
#     text = """We propose a layered framework for incorporating example-based skinning algorithms such as Pose Space Deformation or Shape-by-Example into an existing character animation system.
#     The challenge in implementing example-based skinning in an existing system lies in the fact that it is generally believed that the interpolation of the examples is best performed before doing other skinning deformations.
#     It is therefore necessary to invert the operation of these skinning and deformation operators."""

#     sentence_embeddings, sentence_matrix = layer_2_sentence_level(text)

#     print(f"Number of sentences: {sentence_embeddings.shape[0]}")
#     print("Sentence-Level Adjacency Matrix:")
#     print(sentence_matrix)

In [15]:
import spacy
import pickle
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import os

# Load SpaCy model globally
nlp = spacy.load("en_core_web_sm")

# Load clean_data
with open("clean_data_list.pkl", "rb") as f:
    clean_data = pickle.load(f)

# Function to process a single sample
def process_sample(sample):
    """
    Process a single sample sequentially.
    Args:
        sample (list): A list containing [text, label].

    Returns:
        list: Processed sample with additional layers.
    """
    try:
        text = sample[0]
        label = sample[1]

        # Perform Layer 1, Layer 2, and Layer 3 processing (sequential)
        word_level = layer_1(text, nlp)
        sentence_level = layer_2_sentence_level(text, nlp)
        concept_level = layer_3_concept_level(text, nlp)

        # Construct result dictionary
        layer_dict = {
            "layer_1": word_level,
            "layer_2": sentence_level,
            "layer_3": concept_level,
        }
        return [text, label, layer_dict]
    except Exception as e:
        return {"error": str(e), "sample": sample}

# Process all samples for a single ID
def process_samples_for_id(id_samples):
    """
    Process all samples for a given ID sequentially.
    Args:
        id_samples (tuple): A tuple containing (ID, list of samples).

    Returns:
        tuple: (ID, results for all samples).
    """
    id, samples = id_samples
    results = []

    # Process samples in parallel
    with ProcessPoolExecutor() as executor:
        results = list(executor.map(process_sample, samples))

    # Save results for the ID to a pickle file
    results_dir = "data_2"
    os.makedirs(results_dir, exist_ok=True)
    save_path = os.path.join(results_dir, f"results_{id}.pkl")
    with open(save_path, "wb") as f:
        pickle.dump(results, f)

    return id, save_path

# Parallelize processing across IDs
def parallel_process_ids(clean_data, max_workers=2):
    """
    Processes all IDs in parallel.

    Args:
        clean_data (dict): Dictionary of ID -> list of samples.
        max_workers (int): Number of parallel workers.

    Returns:
        list: List of processed IDs with their corresponding pickle paths.
    """
    id_sample_pairs = list(clean_data.items())
    processed_ids = []

    # Process IDs in parallel
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        for id, save_path in tqdm(executor.map(process_samples_for_id, id_sample_pairs), total=len(id_sample_pairs)):
            processed_ids.append((id, save_path))

    return processed_ids

# Run the processing
if __name__ == "__main__":
    processed_ids = parallel_process_ids(clean_data)
    print(f"Processing completed. Results saved to pickle files: {processed_ids}")

100%|██████████| 40/40 [23:25<00:00, 35.13s/it]

Processing completed. Results saved to pickle files: [('A01', 'data_2/results_A01.pkl'), ('A02', 'data_2/results_A02.pkl'), ('A03', 'data_2/results_A03.pkl'), ('A04', 'data_2/results_A04.pkl'), ('A05', 'data_2/results_A05.pkl'), ('A06', 'data_2/results_A06.pkl'), ('A07', 'data_2/results_A07.pkl'), ('A08', 'data_2/results_A08.pkl'), ('A09', 'data_2/results_A09.pkl'), ('A10', 'data_2/results_A10.pkl'), ('A11', 'data_2/results_A11.pkl'), ('A12', 'data_2/results_A12.pkl'), ('A13', 'data_2/results_A13.pkl'), ('A14', 'data_2/results_A14.pkl'), ('A15', 'data_2/results_A15.pkl'), ('A16', 'data_2/results_A16.pkl'), ('A17', 'data_2/results_A17.pkl'), ('A18', 'data_2/results_A18.pkl'), ('A19', 'data_2/results_A19.pkl'), ('A20', 'data_2/results_A20.pkl'), ('A21', 'data_2/results_A21.pkl'), ('A22', 'data_2/results_A22.pkl'), ('A23', 'data_2/results_A23.pkl'), ('A24', 'data_2/results_A24.pkl'), ('A25', 'data_2/results_A25.pkl'), ('A26', 'data_2/results_A26.pkl'), ('A27', 'data_2/results_A27.pkl'), (

In [4]:
# Load clean_data

with open("labeled_directory.pkl", "rb") as f:
    clean_data = pickle.load(f)



AttributeError: Can't get attribute 'nested_defaultdict' on <module '__main__'>

In [2]:
import pickle
with open("labeled_directory.pkl", "rb") as f:
    clean_data = pickle.load(f)

AttributeError: Can't get attribute 'nested_defaultdict' on <module '__main__'>

In [3]:
import dill as pickle

In [4]:
with open("clean_data_list.pkl","rb") as f:
    x = pickle.load(f)

x["A05"][20]

['n . \uf8ee \uf8ef \uf8f0 (T (T 2,e 2,e 1 k − − . . . T T 1,e 1,e k 1 )v )v d d ··· ··· .. . (T (T n,e n,e 1 k − − . . . T T 1,e 1,e k 1 )v )v d d \uf8f9 \uf8fb \uf8fa \uf8f0 \uf8ee \uf8ef \uf8ef w w w . . . 2 n 3 \uf8fa \uf8fb \uf8fa \uf8f9 = \uf8ef \uf8f0 \uf8ee v v e e 1 k − − T T . . . 1,e 1,e 1 k v v d d \uf8f9 \uf8fa \uf8fb 5 To appear at SIGGRAPH 2003 The matrix used to solve for vertex positions is as follows. \uf8ee \uf8f0 \uf8ef ∑ ∑ i=1 n i=1 n w w . . . i i T T i,e i,e 1 k \uf8f9 \uf8fa \uf8fb v d = \uf8f0 \uf8ef \uf8ee v v . . . e e 1 k \uf8fa \uf8fb \uf8f9 To handle homogeneous coordinates, the translation parts of the ∑ i=1 n w i T i,e k matrices are subtracted from the v e on the right hand side. We solve these least-squares problems using the singular value decomposition. This lets us detect when our matrices are rank deficient, leading to overfitting. We detect this by comparing the ratio of the largest singular value to the smallest, and issuing a warning if there ar

3

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from pprint import pprint


# Example function with error handling
def process_sample(sample):
    try:
        text = sample[0]
        word_level = layer_1(text,nlp)
        #(l2_embeddings,l2_matrix,l2_masks) = layer_2_sentence_level(text)
        #concept_level = layer_3_concept_level(text)
        sample.append({'layer_1': word_level
        #'layer_2': (l2_embeddings,l2_matrix),
        #'layer_3': concept_level,
        #'masks': l2_masks
        })
        return  sample
    except Exception as e:
        pprint(f"Error processing sample {e}")
        return None  # Or handle as needed

# Parallelize the processing using ProcessPoolExecutor

DATA_DIR = "./data/"
remaining_keys = list(clean_data.keys())
for id in tqdm(remaining_keys):
  samples = clean_data[id]
  with ProcessPoolExecutor() as executor:
      results = list(tqdm(executor.map(process_sample, samples), total=len(samples)))
  with open(DATA_DIR + str(id)+"layer1.pkl","wb") as paper:
    pickle.dump(results,paper)

In [19]:
import spacy
import pickle
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor
import os

# Load SpaCy model globally
nlp = spacy.load("en_core_web_sm")

# Load clean_data
with open("clean_data_list.pkl", "rb") as f:
    clean_data = pickle.load(f)

# Function to process a single sample
def process_sample(sample):
    try:
        text = sample[0]
        label = sample[1]

        # Perform Layer 1 processing (sequentially)
        word_level = layer_1(text, nlp)

        # Construct result dictionary
        layer_dict = {'layer_1': word_level}
        return [text, label, layer_dict]
    except Exception as e:
        return {"error": str(e), "sample": sample}

# Process all samples for a single ID
def process_id(id_samples):
    """
    Process all samples for a given ID.
    Args:
        id_samples (tuple): A tuple containing (ID, list of samples).

    Returns:
        tuple: (ID, results for all samples).
    """
    id, samples = id_samples
    results = []

    # Process each sample sequentially (inner operations are sequential)
    for sample in samples:
        result = process_sample(sample)
        if result:
            results.append(result)

    # Save results for the ID to a pickle file
    save_path = f"results_{id}.pkl"
    with open(save_path, "wb") as f:
        pickle.dump(results, f)

    return id, save_path

# Parallelize processing across IDs
def parallel_process_ids(clean_data, max_workers=2):
    """
    Process all IDs in parallel.
    Args:
        clean_data (dict): Dictionary of ID -> list of samples.
        max_workers (int): Number of parallel workers.

    Returns:
        list: List of processed IDs with their corresponding pickle paths.
    """
    id_sample_pairs = list(clean_data.items())
    processed_ids = []

    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        # Parallelize the processing of IDs
        for id, save_path in tqdm(executor.map(process_id, id_sample_pairs), total=len(id_sample_pairs)):
            processed_ids.append((id, save_path))

    return processed_ids

# Run the processing
if __name__ == "__main__":
    processed_ids = parallel_process_ids(clean_data)
    print(f"Processing completed. Results saved to pickle files: {processed_ids}")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
 10%|█         | 4/40 [05:07<46:04, 76.80s/it]


KeyboardInterrupt: 

In [18]:
nlp = spacy.load("en_core_web_sm")

import pickle
import warnings


with open("clean_data_list.pkl","rb") as f:
      clean_data = pickle.load(f)


from tqdm import tqdm
from pprint import pprint


# Example function with error handling
def process_samples(sample,nlp=nlp):
    #try:
    text = sample[0]
    word_level = layer_1(text, nlp)
    #(l2_embeddings,l2_matrix,l2_masks) = layer_2_sentence_level(text,nlp)
    #concept_level = layer_3_concept_level(text,nlp)
    layer_dict = {'layer_1': word_level}
    #'layer_2': (l2_embeddings,l2_matrix),
    #'layer_3': concept_level}
        #'masks': l2_masks
    return [sample[0],sample[1],layer_dict]
        #except Exception as e:
            #print(f"Problem loading sample {e}")
    
# Parallelize the processing using ProcessPoolExecutor

        

from collections import defaultdict
test_results = defaultdict(list)
for id in tqdm(clean_data.keys()):
     samples = clean_data[id]
     results = []
     for sample in tqdm(samples):
         results.append(process_samples(sample))
     test_results[id] = results



  2%|▎         | 1/40 [02:41<1:44:58, 161.50s/it]


KeyboardInterrupt: 

In [4]:
with open("./data_2/results_A01.pkl","rb") as f:
      test = pickle.load(f)

2

In [ ]:
nlp = spacy.load("en_core_web_sm")

import pickle
import warnings


with open("clean_data_list.pkl","rb") as f:
      clean_data = pickle.load(f)


from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from pprint import pprint


# Example function with error handling
def process_samples(samples,nlp=nlp):
    new_samples = []
    for sample in samples:
    #try:
        text = sample[0]
        word_level = layer_1(text,nlp)
        (l2_embeddings,l2_matrix,l2_masks) = layer_2_sentence_level(text,nlp)
        concept_level = layer_3_concept_level(text,nlp)
        layer_dict = {'layer_1': word_level
        #'layer_2': (l2_embeddings,l2_matrix),
        #'layer_3': concept_level}
            #'masks': l2_masks
        new_samples.append([sample[0],sample[1],layers])
    return new_samples
        #except Exception as e:
            #print(f"Problem loading sample {e}")
    
# Parallelize the processing using ProcessPoolExecutor

from collections import defaultdict
test_results = defaultdict(list)
for id in tqdm(clean_data.keys()):
     samples = clean_data[id]
     results = []
     with ProcessPoolExecutor() as executor:
        results.append(list(tqdm(executor.map(process_samples, samples), total=len(samples))))
     test_results[id] = results



In [68]:
nlp = spacy.load("en_core_web_sm")

import pickle
import warnings


with open("clean_data_list.pkl","rb") as f:
      clean_data = pickle.load(f)


from tqdm import tqdm
from pprint import pprint


# Example function with error handling
def process_sample(sample,nlp=nlp):
    try:
        text = sample[0]
        word_level = layer_1(text,nlp)
        (l2_embeddings,l2_matrix,l2_masks) = layer_2_sentence_level(text,nlp)
        concept_level = layer_3_concept_level(text,nlp)
        layer_dict = {'layer_1': word_level,
        'layer_2': (l2_embeddings,l2_matrix),
        'layer_3': concept_level,
        'masks': l2_masks
        })
        return [text,sample[1],layer_dict]
    except Exception as e:
        pprint(f"Error processing sample {e}")
        return None  # Or handle as needed

# Parallelize the processing using ProcessPoolExecutor

from collections import defaultdict
test_results = defaultdict(list)
#remaining_keys = list()
for id in tqdm(clean_data.keys()):
  samples = clean_data[id]
  results = []
  for [sample,label] in samples:
      results = process_sample(sample)
      results.append(sample)
  test_results[id] = results


#emb,adj,tokens = layer_1(x,nlp)

100%|██████████| 40/40 [08:56<00:00, 13.42s/it]


In [52]:
for id,samples in clean_data.items():
    print(id)
    for (label,sample) in samples:
        print(sample)
        

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [85]:
type(clean_data["A01"][0])

list

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict
import spacy
from tqdm import tqdm
import pickle

# Load SpaCy globally
nlp_global = spacy.load("en_core_web_sm")

# Load clean_data
with open("clean_data_list.pkl", "rb") as f:
    clean_data = pickle.load(f)

# Worker initialization
def init_worker():
    """
    Initializes worker with a SpaCy model to avoid pickling issues.
    """
    global nlp_worker
    nlp_worker = spacy.blank("en")  # Lightweight alternative for multiprocessing

# Process a single sample
def process_sample(sample):
    try:
        global nlp_worker
        text, label = sample[0], sample[1]
        layer_1_output = layer_1(text, nlp_worker)
        layer_2_output = layer_2_sentence_level(text, nlp_worker)
        layer_3_output = layer_3_concept_level(text, nlp_worker)

        return {
            "text": text,
            "label": label,
            "layer_1": layer_1_output,
            "layer_2": layer_2_output,
            "layer_3": layer_3_output,
        }
    except Exception as e:
        return {"error": str(e), "text": sample[0], "label": sample[1]}

# Process data for a single ID
def process_data_for_id(data_for_id):
    """
    Process samples for a single ID in parallel.
    """
    id, samples = data_for_id
    results = []
    for sample in samples:
        result = process_sample(sample)
        if result and "error" not in result:
            results.append(result)
    return id, results

# Parallel processing
def process_samples_in_parallel(clean_data, max_workers=2, batch_size=50):
    """
    Processes clean_data in parallel.

    Args:
        clean_data (dict): Dictionary of {id: list of samples}.
        max_workers (int): Number of worker processes.
        batch_size (int): Number of IDs processed per batch.

    Returns:
        dict: Results for all IDs.
    """
    test_results = defaultdict(list)
    id_sample_pairs = list(clean_data.items())
    batch_size = 5
    # Use ProcessPoolExecutor for parallel processing
    with ProcessPoolExecutor(max_workers=max_workers, initializer=init_worker) as executor:
        # Batch process IDs
        for i in tqdm(range(0, len(id_sample_pairs), batch_size), desc="Processing Batches"):
            batch = id_sample_pairs[i : i + batch_size]
            for id, results in executor.map(process_data_for_id, batch):
                test_results[id].extend(results)

    return test_results

# Execute the processing
test_results = process_samples_in_parallel(clean_data)


Processing Batches:   0%|          | 0/8 [00:00<?, ?it/s]

0it [00:00, ?it/s]